In [ ]:
## Necessary packages
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import warnings
warnings.filterwarnings("ignore")

# 1. TimeGAN model
from timegan import timegan
# 2. Data loading
from data_loading import real_data_loading, sine_data_generation, real_data_loading_sce1
# 3. Metrics
from metrics.discriminative_metrics import discriminative_score_metrics
from metrics.predictive_metrics import predictive_score_metrics
from metrics.visualization_metrics import visualization

import tensorflow as tf

import os, time

In [ ]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = "2"
config = tf.compat.v1.ConfigProto() # Another Version: config = tf.ConfigProto()
config.gpu_options.allow_growth = True

In [ ]:
## Data loading
data_name = 'stock'
seq_len = 24
per_test = 0.05 # 5% as testing

if data_name in ['stock', 'energy']:
    train_data, test_data = real_data_loading_sce1(data_name, seq_len, per_test)
elif data_name == 'sine':
    # Set number of samples and its dimensions
    no, dim = 10000, 5
    train_data, test_data = sine_data_generation(no, seq_len, dim)
    
print(data_name + ' dataset is ready.')

In [ ]:
np.array(train_data).shape, np.array(test_data).shape

In [ ]:
## Newtork parameters
parameters = dict()

parameters['module'] = 'gru' 
parameters['hidden_dim'] = 24
parameters['num_layer'] = 3
parameters['iterations'] = 5000
parameters['batch_size'] = 128

In [ ]:
# Run TimeGAN
start = time.time()
generated_data = timegan(train_data, parameters)   
end = time.time()
print('Finish Synthetic Data Generation')
print(f"Runtime of the Generator is {end - start}")

In [ ]:
filename = data_name + "_data_TimeGAN_5000_sce1-wTest.npy"
np.save(filename, generated_data)

In [ ]:
generated_data.shape

In [ ]:
generated_data[0][0]

In [ ]:
metric_iteration = 5

predictive_score = list()
time_usage = list()
for tt in range(metric_iteration):
    start = time.time()
    temp_pred = predictive_score_metrics(test_data, generated_data)
    end = time.time()
    predictive_score.append(temp_pred)
    time_usage.append(end-start)
    
print('Predictive score: ' + str(np.round(np.mean(predictive_score), 4)))

In [ ]:
print("all the predictive scores: ", predictive_score)
print('Predictive score avg. value: ' + str(np.round(np.mean(predictive_score), 4)))
print('Predictive score std. value: ' + str(np.round(np.std(predictive_score), 4)))
#================Time calculation==========================#
print('Time usage avg. value: ' + str(np.round(np.mean(time_usage), 4)))
print('Time usage std. value: ' + str(np.round(np.std(time_usage), 4)))

In [ ]:
#ori_data = np.append(train_data, test_data, axis=0) # append the train and test data together

In [ ]:
visualization(train_data, generated_data, 'pca')
visualization(train_data, generated_data, 'tsne')

In [ ]:
!mv ./pca-results-stock-sce1-wTest.png ./figures_timegan_sce1_wTest
!mv ./tsne-results-stock-sce1-wTest.png ./figures_timegan_sce1_wTest